In [1]:
#请 修改->笔记本设置->T4 GPU,然后确认下方输出Tesla T4来确认有显卡
#然后点击 代码执行工具->全部运行 等待约五到十分钟
#最后点击 最后下方显示的的链接
# !nvidia-smi

zsh:1: command not found: nvidia-smi


In [ ]:
!pip install gradio
!pip install huggingface_hub
!pip install pynvml
!pip install rwkv
!pip install Ninja

In [ ]:
import gradio as gr
import os, gc, copy, torch
from datetime import datetime
from huggingface_hub import hf_hub_download
from pynvml import *

# Flag to check if GPU is present
HAS_GPU = False

# Model title and context size limit
ctx_limit = 2000
title = "RWKV-5-H-World-3B"
model_file = "rwkv-5-h-world-3B"

# Get the GPU count
# try:
#     nvmlInit()
#     GPU_COUNT = nvmlDeviceGetCount()
#     if GPU_COUNT > 0:
#         HAS_GPU = True
#         gpu_h = nvmlDeviceGetHandleByIndex(0)
# except NVMLError as error:
#     print(error)


os.environ["RWKV_JIT_ON"] = '1'

# Model strat to use
MODEL_STRAT="cpu bf16"
os.environ["RWKV_CUDA_ON"] = '0' # if '1' then use CUDA kernel for seq mode (much faster)

# Switch to GPU mode
if HAS_GPU == True :
    os.environ["RWKV_CUDA_ON"] = '1'
    MODEL_STRAT = "cuda bf16"

# Load the model accordingly
from rwkv.model import RWKV
model_path = hf_hub_download(repo_id="a686d380/rwkv-5-h-world", filename=f"{model_file}.pth")
model = RWKV(model=model_path, strategy=MODEL_STRAT)
from rwkv.utils import PIPELINE, PIPELINE_ARGS
pipeline = PIPELINE(model, "rwkv_vocab_v20230424")


In [6]:

# Prompt generation
def generate_prompt(instruction, input=""):
    instruction = instruction.strip().replace('\r\n','\n').replace('\n\n','\n')
    input = input.strip().replace('\r\n','\n').replace('\n\n','\n')
    if input:
        return f"""Instruction: {instruction}
Input: {input}
Response:"""
    else:
        return f"""User: hi
Assistant: Hi. I am your assistant and I will provide expert full response in full details. Please feel free to ask any question and I will always answer it.
User: {instruction}
Assistant:"""

# Evaluation logic
def evaluate(
    ctx,
    token_count=500,
    temperature=1.0,
    top_p=0.7,
    presencePenalty = 0.1,
    countPenalty = 0.1,
):
    print(ctx)
    args = PIPELINE_ARGS(temperature = max(0.2, float(temperature)), top_p = float(top_p),
                     alpha_frequency = countPenalty,
                     alpha_presence = presencePenalty,
                     token_ban = [], # ban the generation of some tokens
                     token_stop = [0]) # stop generation whenever you see any token here
    ctx = ctx.strip()
    all_tokens = []
    out_last = 0
    out_str = ''
    occurrence = {}
    state = None
    for i in range(int(token_count)):
        out, state = model.forward(pipeline.encode(ctx)[-ctx_limit:] if i == 0 else [token], state)
        for n in occurrence:
            out[n] -= (args.alpha_presence + occurrence[n] * args.alpha_frequency)

        token = pipeline.sample_logits(out, temperature=args.temperature, top_p=args.top_p)
        if token in args.token_stop:
            break
        all_tokens += [token]
        for xxx in occurrence:
            occurrence[xxx] *= 0.996
        if token not in occurrence:
            occurrence[token] = 1
        else:
            occurrence[token] += 1

        tmp = pipeline.decode(all_tokens[out_last:])
        if '\ufffd' not in tmp:
            out_str += tmp
            yield out_str.strip()
            out_last = i + 1

    if HAS_GPU == True :
        gpu_info = nvmlDeviceGetMemoryInfo(gpu_h)
        print(f'vram {gpu_info.total} used {gpu_info.used} free {gpu_info.free}')

    del out
    del state
    gc.collect()

    if HAS_GPU == True :
        torch.cuda.empty_cache()

    yield out_str.strip()


##########################################################################

# Gradio blocks
with gr.Blocks(title=title) as demo:
    gr.HTML(f"<div style=\"text-align: center;\">\n<h1>RWKV-5 World v2 - {title}</h1>\n</div>")
    with gr.Tab("Raw Generation"):
        gr.Markdown(f"This is RWKV-5-h ")
   with gr.Row():
            with gr.Column():
                prompt = gr.Textbox(lines=2, label="Prompt", value="""写一段小说""")
                token_count = gr.Slider(10, 2048, label="Max Tokens", step=10, value=200)
                temperature = gr.Slider(0.2, 2.0, label="Temperature", step=0.1, value=1.0)
                top_p = gr.Slider(0.0, 1.0, label="Top P", step=0.05, value=0.3)
                presence_penalty = gr.Slider(0.0, 1.0, label="Presence Penalty", step=0.1, value=1)
                count_penalty = gr.Slider(0.0, 1.0, label="Count Penalty", step=0.1, value=1)
            with gr.Column():
                with gr.Row():
                    submit = gr.Button("Submit", variant="primary")
                    clear = gr.Button("Clear", variant="secondary")
                output = gr.Textbox(label="Output", lines=5)
        data = gr.Dataset(components=[prompt, token_count, temperature, top_p, presence_penalty, count_penalty], label="Example Instructions", headers=["Prompt", "Max Tokens", "Temperature", "Top P", "Presence Penalty", "Count Penalty"])
        submit.click(evaluate, [prompt, token_count, temperature, top_p, presence_penalty, count_penalty], [output])
        clear.click(lambda: None, [], [output])
        data.click(lambda x: x, [data], [prompt, token_count, temperature, top_p, presence_penalty, count_penalty])

# Gradio launch
demo.queue(max_size=10)
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7861

Could not create share link. Missing file: /Users/fcfu/PycharmProjects/LLM_playground/.venv/lib/python3.11/site-packages/gradio/frpc_darwin_arm64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.3
3. Move the file to this location: /Users/fcfu/PycharmProjects/LLM_playground/.venv/lib/python3.11/site-packages/gradio



点击弹出的链接



高义拍完了照片，赤裸裸的走到白洁身边，把她抱到卧室的床上，扒下她的裙子胸罩，白洁只穿着白色的丝袜，仰躺在床上，一对雪白丰满的乳房在胸前隆起着，即使躺着也那么挺实，高义光着身子躺在白洁身边，双手不停地抚摸着白洁全身，很快阴茎又硬了。
高义拍完了照片，赤裸裸的走到白洁身边，把她抱到卧室的床上，扒下她的裙子胸罩，白洁只穿着白色的丝袜，仰躺在床上，一对雪白丰满的乳房在胸前隆起着，即使躺着也那么挺实，高义光着身子躺在白洁身边，双手不停地抚摸着白洁全身，很快阴茎又硬了。
女主人公：杏梅，30岁，是一个漂亮的日本妓女，所以身材很好，乳房很坚挺，阴毛浓密，喜欢口交，乳交，并且喜欢性交的时候被别人用力抓住乳房，叫床声音很大。

男主人公：我，40岁，成熟健壮，喜欢亲吻杏梅的小穴，玩弄她的乳房。喜欢捏住她的奶子猛插

场景：在我的宾馆房间

故事背景：杏梅收到我的巨额资金，今天回用全身心服务我。

条件和约束
要有人物性格和细节描写

对于性爱场景，要生动，富有想象力

突出杏梅的性技巧非常好的重点
我会多次射精

人物有对话和情感交流

做爱场景要描写的露骨且细致

不少于4000字

只用中文写小说

#参考下面的写法：
“每一次都把我的肉茎完全地吞进小肉洞中，溢出的大量蜜汁也顺着我的肉棒流到了我的阴囊和大腿上。"啊……爸你的大肉棒…喔…干的我…我好爽……喔……妹妹的小穴……喔……快…快受不了了……啊……不…不行了…我要死了……喔……干…干死人了……呀……亲哥哥……妹妹爱…爱死你了……呀……" 我一手抱着月月的香肩，一手还停留在她的乳房上轻轻的搓揉，大肉棒在那一张一合的小穴里，是愈抽愈急，愈插愈猛。”





AI
杏梅已经躺在床上，她浑身酥麻、软绵绵地躺卧着。她用她那柔弱无骨的手臂搂住我的脖子，头枕在我的胸口上。她的呼吸很急促，眼睛紧紧地闭着。

我吻了吻杏梅的鼻子，在她的耳边轻轻地说道：“我终于把你弄到手了！”
杏梅微微睁开了一只眼睛，向我望来。“啊……”她轻声叫道。
“怎么？你害怕了？”我的舌头从杏梅的小嘴边缘滑入，伸进那甜美的小嘴，开始吸吮起来。随后又用舌尖去舔她柔软的唇、乳头和阴蒂。
杏梅紧闭着双眼，全身发抖，浑身打起了冷颤。

我继续吻她的脖子、手臂，直至到达她的胸部。“啊……”杏梅无力地呻吟了一声。

“嗯……”我的手伸进了她的衣服里面，“啊……”杏梅又一次轻轻地叫